In [1]:
# Import necessary libraries
import getpass
import glob
import os
import pandas as pd
import numpy as np
import h5py


In [6]:

# Get the current user
user = getpass.getuser()
print("Hello", user)

# Define base directory paths
base = 'C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
print('base = ', base)

# Define paths for saving filtered files and results
files_savepath = base + '\\all_data_filtered_mat\\'
print('filtered files will be stored at', files_savepath)

savepath = base + '\\results\\'
print('plots and stuff will be at', savepath)

files = glob.glob(r'C:\\Users\\{}\\Dropbox\\CPLab\\all_data_mat\\*.mat'.format(user))
print(files)


Hello Thomas
base =  C:\Users\Thomas\Dropbox\CPLab
filtered files will be stored at C:\Users\Thomas\Dropbox\CPLab\all_data_filtered_mat\
plots and stuff will be at C:\Users\Thomas\Dropbox\CPLab\results\
['C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230529_dk1_nocontext.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230529_dk3_nocontext.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230529_dk5_nocontext.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230529_dk6_nocontext.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230531_dk1_nocontext_day2.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230531_dk3_nocontext_day2.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230531_dk5_nocontext_day2.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\20230531_dk6_nocontext_day2.mat', 'C:\\\\Users\\\\Thomas\\\\Dropbox\\\\CPLab\\\\all_data_mat\\2023

In [ ]:

"""Load and clean filter data from an Excel file"""

filter_data = pd.read_excel("C:\\Users\\{}\\Dropbox\\CPLab\\filecheck.xlsx".format(user)) #Loading up the data that contains the manual annotation from Christiane
filter_data = filter_data.dropna(axis=1, how='all') #Removes the columns where all the values are NA
filter_data = filter_data.dropna(axis=0, how='all') # Removes the rows where all the values are NA
filter_data = filter_data.drop(filter_data.columns[-1], axis=1) #Drop the last column that just contains comments
filter_data.reset_index(drop=True, inplace=True) # Reset the index becuase of the dropped rows
filter_data.columns = ['file', 'LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp', 'date'] #Manually set the column names

"Extract file names without extensions from filter data"

filter_data_files = list(filter_data['file']) #take the 'file' column from the dataframe
filter_base_names_without_ext = [os.path.splitext(os.path.basename(file))[0] for file in filter_data_files] #extract the file names, removes the paths from the filenames and stores it as a list
print(filter_base_names_without_ext)

# Update filter data with base names without extensions
filter_data['file'] = filter_base_names_without_ext

# Extract annotations from filter data
filter_data_annotations = filter_data.iloc[:, 1:6] #Extracting annotations "no" and "ok"
filter_data_annotations = np.array(filter_data_annotations) #saving it as an array
print(np.unique(filter_data_annotations)) #printing out the uniques annotations to make sure that annotations are "no" and "ok" only


In [ ]:

# Find all .mat files in the specified directory
files = glob.glob(r'C:\\Users\\{}\\Dropbox\\CPLab\\all_data_mat\\*.mat'.format(user))
print(files)

# Print the number of files and the shape of the filter data
print(len(files))
print(filter_data.shape)

# Extract base names without extensions from the list of files
files_list = []
for file in files:
    base_name = os.path.basename(file)
    base_name_without_ext = os.path.splitext(base_name)[0]
    files_list.append(base_name_without_ext)


In [ ]:

# Find uncommon elements between the list of files and filter data
set1 = set(files_list)
set2 = set(filter_data['file'])
uncommon_elements = set1.symmetric_difference(set2)
uncommon_elements_list = list(uncommon_elements)
print(uncommon_elements_list)

# These two files don't exist and are a typo in the filecheck.xlsx file
# ['20230609_dk1_BW_nocontext_day2', '20230822_dk1_BW_nocontext_os2_day2']


In [ ]:
"""Here we will drop the data file columns that are "NO" in the manual annotation file(filecheck.xlsx)"""

# Define channel names
channel_names = ['LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp']

# Initialize a list to store files that are not found
not_here_list = []

# Iterate over each file and delete specified columns if necessary
for file in files: #Loading the data files
    f = h5py.File(file, 'r+') #reading the file with the option of writing to it by using "r+" 
    try:
        base_name = os.path.basename(file)
        base_name_without_ext = os.path.splitext(base_name)[0]
        if base_name_without_ext in filter_base_names_without_ext: #checking if the data file exists in filecheck.xlsx 
            print(base_name_without_ext, 'yes')
            row = filter_data.loc[filter_data['file'] == base_name_without_ext] #locates the row of the file in filecheck.xlsx
            for column in row.columns: #Iterating through the manual annotations
                value = row[column].values[0]
                if value == 'no' or value == 'NO': #If the annotation of a channel is "NO"
                    print(f"Column '{column}' has value 'no'")
                    if column in f.keys():
                        del f[column] # Deletes that channel in the data file
                        print(f"Deleted key '{column}' from file '{file}'")
                elif value == 'ok' or value == 'OK': # If the annotation of a channel is  "OK"
                    print(f"Column '{column}' has value 'ok'") #Do nothing
        else:
            print(base_name_without_ext, 'this file not here') #if the data file does not exist in filecheck.xlsx
            not_here_list.append(base_name_without_ext) 
    finally:
        f.close() #Close the data file. This saves the manipulations we have made in the above code.

print(not_here_list)


In [5]:

# Check the channels in each file
for file in files:
    f = h5py.File(file, 'r')
    channels = f.keys()
    print(file, channels)
    f.close()


NameError: name 'files' is not defined

===================================================== Below Code will Select Channels based on Threshold value=======================================


In [1]:
# Import necessary libraries
import getpass
import glob
import os
import pandas as pd
import numpy as np
import h5py


In [2]:
#Fetch the current user
user= (getpass.getuser())
print("Hello", user)

if user == 'CPLab':
    base='D:\\Dropbox\\CPLab'
else:
    base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
savepath=base+'\\results\\'
#Set the basepath, savepath and load the data files
files = glob.glob(base+'\\all_data_mat_250825\\*.mat')
print(files)
# Extract base names without extensions from the list of files
files_list = []
for file in files:
    base_name = os.path.basename(file)
    base_name_without_ext = os.path.splitext(base_name)[0]
    files_list.append(base_name_without_ext)
print(files_list)
manual_check = pd.read_excel(savepath+'baseline_power_analysis_with_spectrograms.xlsx') #Loading the manual check file
print(manual_check)
manual_check_file_list = manual_check['base_name'].unique().tolist()
print(manual_check_file_list)

Hello CPLab
['D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk1_nocontext.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk3_nocontext.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk5_nocontext.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk6_nocontext.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk1_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk3_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk5_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk6_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230609_dk1_BW_nocontext_day1.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230609_dk3_BW_nocontext_day1.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230610_dk1_BW_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230610_dk3_BW_nocontext_day2.mat', 'D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230615_dk5_BW_context_day1.mat', 'D:\\D

In [3]:

# Find uncommon elements between the list of files and filter data
set1 = set(files_list)
set2 = set(manual_check_file_list)
uncommon_elements = set1.symmetric_difference(set2)
uncommon_elements_list = list(uncommon_elements)
print(uncommon_elements_list)


['20230529_dk3_nocontext', '20230529_dk1_nocontext', '20230719_dk1_nocontext_os2_day2_part1', '20230531_dk3_nocontext_day2', '20230529_dk5_nocontext', '20230531_dk1_nocontext_day2', '20230718_dk1_nocontext_os2', '20230718_dk5_nocontext_os2', '20230531_dk5_nocontext_day2', '20230529_dk6_nocontext', '20230719_dk5_nocontext_os2_day2', '20230531_dk6_nocontext_day2', '20230719_dk1_nocontext_os2_day2_part2', '20230718_dk6_nocontext_os2', '20230719_dk6_nocontext_os2_day2']


In [5]:
import importlib
import lfp_pre_processing_functions
importlib.reload(lfp_pre_processing_functions)
# Define channel names
channel_names = ['LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp']

# Initialize a list to store files that are not found
not_here_list = []

threshold = 0.2
for file in files:
    f=h5py.File(file, 'r+')
    #print(file, f.keys())
    try:
        date, rat_id, task = lfp_pre_processing_functions.exp_params(file)
        if task == 'nocontextday2' or task == 'nocontextos2':
            task = 'nocontext'
        if task =='nocontext':
            continue
        print(date, rat_id, task)
        base_name = os.path.basename(file)
        base_name_without_ext = os.path.splitext(base_name)[0]
        if base_name_without_ext in manual_check_file_list: # Check if the Manual annotation of the file exists
            print("File found in manual check list:", base_name_without_ext)
            for channel in channel_names:
                
                if channel in f.keys():
                    manual_check_file_channel = manual_check[(manual_check['base_name'] == base_name_without_ext) & (manual_check['channel_id'] == channel)]
                    baseline_power_ratio = manual_check_file_channel['line_power_ratio'].values[0]
                    print("Channel found:", channel, baseline_power_ratio)
                    if baseline_power_ratio > threshold:
                        print(f"Baseline power ratio is greater than {threshold}")
                        del f[channel]
                        print(f"Deleted key '{channel}' from file '{base_name_without_ext}'")
                else:
                    print("Channel not found:", channel)
        else:
            print('File not found in manual check list:', base_name_without_ext)
            not_here_list.append(base_name_without_ext)
    finally:
        f.close()
        print('Closed the file')


D:\Dropbox\CPLab\all data mat250825\20230529
File not found in manual check list: 20230529_dk1_nocontext
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230529
File not found in manual check list: 20230529_dk3_nocontext
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230529
File not found in manual check list: 20230529_dk5_nocontext
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230529
File not found in manual check list: 20230529_dk6_nocontext
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230531
File not found in manual check list: 20230531_dk1_nocontext_day2
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230531
File not found in manual check list: 20230531_dk3_nocontext_day2
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230531
File not found in manual check list: 20230531_dk5_nocontext_day2
Closed the file
D:\Dropbox\CPLab\all data mat250825\20230531
File not found in manual check list: 20230531_dk6_nocontext_day2
Closed the file
D:\Dropbox\C

==================================================== End of Main Code ==============================================================

### The below code is to check for Keyboard annotations in data files.

In [7]:

# Define a function to convert decimal numbers to Unicode characters
def decimal_to_unicode(decimal_list):
    """
    Convert a list of decimal numbers to their corresponding Unicode characters.
    
    Parameters:
    decimal_list (list): List of decimal numbers.
    
    Returns:
    list: List of Unicode characters.
    """
    return [chr(decimal) for decimal in decimal_list]

# Example usage of decimal_to_unicode function
decimal_list = [65, 66, 67, 8364]  # A, B, C, €
unicode_characters = decimal_to_unicode(decimal_list)
print(unicode_characters)

# Define a function to combine strings with an optional separator
def combine_strings(string_list, separator=''):
    """
    Combine strings in a list into a single string with an optional separator.
    
    Parameters:
    string_list (list): List of strings to combine.
    separator (str): Separator to use between strings. Default is an empty string.
    
    Returns:
    str: Combined string.
    """
    return separator.join(string_list)

# Example usage of combine_strings function
string_list = ['Hello', 'world', 'this', 'is', 'a', 'test']
combined_string = combine_strings(string_list, ' ')
print(combined_string)

# Initialize lists to store codes and comments from all files
codes_all = []
comments_all = []

# Iterate over each file and extract codes and comments
for file in files:
    f = h5py.File(file, 'r')
    channels = list(f.keys())

    # Check for the presence of specific channels and extract events
    if 'Keyboard' in channels:
        events = f['Keyboard']
    elif 'keyboard' in channels:
        events = f['keyboard']
    elif 'memory' in channels:
        events = f['memory']
    elif 'Memory' in channels:
        events = f['Memory']

    # Print the title of the events
    print(events['title'][:, 0])
    print(combine_strings(decimal_to_unicode(events['title'][:, 0]), ''))

    # Append codes and comments to the respective lists
    codes_all.append(np.array(events['codes'][0]))
    comments_all.append(np.array(events['comment']))

    # Convert comments to Unicode and combine them into a single string
    please = decimal_to_unicode(events['comment'][:, 0])
    combined_string = combine_strings(please, ' ')
    print(combined_string)

# Concatenate all codes and comments
codes_all = np.concatenate(codes_all)
comments_all = np.concatenate(comments_all)

# Print unique codes and comments
codes_all_unique = np.unique(codes_all)
comments_all_unique = np.unique(comments_all)
print(codes_all_unique)
print(comments_all_unique)

['A', 'B', 'C', '€']
Hello world this is a test
[ 75 101 121  98 111  97 114 100]
Keyboard
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 77 101 109 111 114 121]
Memory
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 77 101 109 111 114 121]
Memory
N o   c o m m e n t
[ 77 101 109 111 114 121]
Memory
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboard
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboard
t o   m a r k   d o o r   o p e n i n g  
[ 77 101 109 111 114 121]
Memory
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboard
N o   c o m m e n t
[ 75 101 121  98 111  97 114 100]
Keyboar